In [1]:
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import cvxpy as cp

(CVXPY) Apr 09 03:15:54 AM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Apr 09 03:15:54 AM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')


## Portfolio Optimization

Portfolio is a collection of investments including stocks. The main objective of portfolio optimization is to minimize the risk while getting the maximum return. However, stocks are a trade-off between risk and return. Higher risk means higher return, so they are expected to do well in the long term but unstable in the short term. According to Investopedia, the best way to allocate investment to assets is to diversify the stocks into different industries. There are different ways that people can allocate their portfolio assets. Personally, I would like moderately lower risk, but enough to get a considerable amount of return. The following 17 stocks are selected across different types of industries and size of market capitalizations.

In [72]:
# get the symbol of the companies
symbols = ["TSLA", "SHOP", "SQ", "QS", "OPEN", "MP", "COIN", "HRI", "CLF", "LOW", "GS", "KO", "SPGI",
           "GIS", "CVS", "COST", "PG"]

keywords:
- Return : the amount the asset is worth after a time period
- Risk : the variance of the returns
- $r_i$: expected return of asset i
- $\sum = [\sigma_{ij}]$: covariance of asset i and j
- $x_i$ : proportion of investment allocated to asset i

### Formulation

Since we would like to have the minimum risk for fixed level of return, we can use Markowitz mean-variance optimization model for portfolio selection.

$$ \text{min } \mathbf{x}^T\Sigma \mathbf{x}$$
$$ \text{s.t.} \quad \mathbf{r}^T\mathbf{x} = R \\ e^T\mathbf{x} = 1 \\ \mathbf{x} \geq 0$$

- $\mathbf{x}^T\Sigma \mathbf{x}$ is the variance (risk) that we minimize
- $\mathbf{r}^T\mathbf{x}$ is the expected return of the portfolio $\sum_i r_i x_i$, so this ensures that the expected return matches the return that we consider
- $R$ is the fixed level of return that we set
- $\mathbf{e}^T\mathbf{x}$ = $\sum_i x_i$, where $x_i$ is the fraction of how much should be allocated to each asset and $\mathbf{e}$ is a vector of 1s.

Source: Math 441 notes

### Compute exepected rates of return for each assets

In [84]:
# get the daily returns
T = 1
R = []
for stock in symbols:
    data = yf.Ticker(stock)
    # get the data of the stock over the 5 years until the present
    df = data.history(period="2y")
    # get the closing value of each day
    close = df['Close'].values
    # calculate return from one closing day to the next closing day
    returns = (close[T:] - close[:-T])/close[:-T]
    R.append(returns)

In [86]:
df = pd.DataFrame(columns = symbols)

for i in range(len(symbols)):
    r = R[i]
    df[symbols[i]] = r.tolist()

df

,TSLA,SHOP,SQ,QS,OPEN,MP,COIN,HRI,CLF,LOW,GS,KO,SPGI,GIS,CVS,COST,PG
0,0.011292,-0.042632,-0.001544,-0.010579,0.030809,-0.047508,-0.026126,-0.001382,0.011494,-0.002199,-0.003055,0.011754,-0.020463,-0.001135,0.001819,-0.005661,-0.003010
1,0.035888,0.023213,0.025714,0.039955,0.090909,0.046824,0.032966,0.019102,0.039610,0.006711,0.006849,0.002633,-0.002517,0.003550,0.003249,0.016737,0.002830
2,-0.036552,-0.041784,-0.037604,-0.014610,0.007991,-0.045702,-0.048453,-0.009847,-0.036852,-0.020583,-0.001025,0.004480,-0.022003,0.003395,-0.012383,-0.001184,-0.005581
3,0.019584,0.002571,-0.015827,-0.057111,-0.066818,-0.050948,-0.014461,0.019136,-0.002594,-0.013712,0.025619,-0.008920,0.007663,-0.001128,-0.002508,-0.014008,-0.009523
4,0.023758,0.042392,0.052182,0.045428,0.041262,0.019970,0.042091,0.047177,0.021782,0.026907,0.018401,0.009776,0.012264,0.012422,0.004254,0.020924,0.014963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,-0.049024,-0.000895,-0.027621,-0.037520,-0.044674,-0.027188,-0.024870,-0.020845,-0.013662,-0.019376,-0.008126,-0.008734,-0.002295,0.006131,-0.072147,-0.013782,-0.000062
495,0.010502,-0.031746,0.005555,0.033898,0.032374,0.024540,0.023349,0.015276,0.020107,-0.015341,0.009387,-0.005320,0.000821,-0.022251,0.010566,-0.007887,-0.027527
496,0.016213,-0.010973,-0.061770,-0.032787,-0.038327,-0.019295,-0.007831,-0.016074,-0.017959,-0.011176,-0.018720,-0.008858,0.004596,0.020148,-0.009383,-0.001148,-0.004483
497,-0.036292,0.006283,0.032651,0.013559,-0.021739,-0.008820,-0.034894,0.014064,-0.007583,0.005504,0.004480,0.003541,0.007399,-0.008383,0.009202,0.012541,0.004181


In [87]:
mean_return = df.mean()
print(mean_return)
np.mean(mean_return)

TSLA   -0.000595
SHOP    0.001356
SQ     -0.000063
QS     -0.000742
OPEN    0.000192
MP     -0.001707
COIN    0.003026
HRI     0.000797
CLF    -0.000125
LOW     0.000577
GS      0.000741
KO      0.000025
SPGI    0.000300
GIS     0.000172
CVS    -0.000434
COST    0.000600
PG      0.000120
dtype: float64


0.0002493778587591997

The R array will be of length 14 with a subarray of length 1225 that represents the daily returns over a period of 5 years. Based on the data, it seems like 5% is a good 

### Compute covarainces for a collection of assets

Covariance measures the relationship between two variables. If it is positive, then both assets move in the same direction (either less return or more return). If it is negative, then one of the asset is going up while the other is going down. This can be a strategy to reduce risk because we know for sure that at least one of the asset will gain and might offset the loss of the other asset. The strategy is called hedging. 

In [88]:
R = np.array(R).T
r = np.mean(R,axis=0) # mean return of each asset
S = np.cov(R.T)

In [89]:
df = pd.DataFrame(index = symbols, columns = symbols)
df = pd.DataFrame(S)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.001331,0.000843,0.000740,0.000972,0.001197,0.000564,0.001188,0.000371,0.000285,0.000206,0.000196,0.000065,0.000235,-0.000007,0.000135,0.000229,0.000043
1,0.000843,0.001996,0.001238,0.001285,0.001725,0.000710,0.001582,0.000532,0.000449,0.000340,0.000305,0.000083,0.000366,-0.000016,0.000101,0.000266,0.000092
2,0.000740,0.001238,0.001683,0.001236,0.001550,0.000734,0.001567,0.000632,0.000571,0.000400,0.000340,0.000118,0.000369,-0.000015,0.000187,0.000259,0.000112
3,0.000972,0.001285,0.001236,0.002931,0.002030,0.000940,0.001856,0.000663,0.000535,0.000393,0.000354,0.000075,0.000325,-0.000026,0.000171,0.000261,0.000076
4,0.001197,0.001725,0.001550,0.002030,0.004736,0.001075,0.002447,0.000819,0.000704,0.000570,0.000434,0.000146,0.000519,-0.000022,0.000165,0.000374,0.000125
5,0.000564,0.000710,0.000734,0.000940,0.001075,0.001270,0.001006,0.000582,0.000541,0.000280,0.000291,0.000076,0.000225,0.000026,0.000121,0.000150,0.000053
6,0.001188,0.001582,0.001567,0.001856,0.002447,0.001006,0.004026,0.000678,0.000669,0.000453,0.000355,0.000095,0.000415,-0.000013,0.000210,0.000364,0.000102
7,0.000371,0.000532,0.000632,0.000663,0.000819,0.000582,0.000678,0.000890,0.000478,0.000270,0.000309,0.000090,0.000234,0.000020,0.000121,0.000180,0.000064
8,0.000285,0.000449,0.000571,0.000535,0.000704,0.000541,0.000669,0.000478,0.000979,0.000224,0.000245,0.000077,0.000211,0.000025,0.000108,0.000146,0.000060
9,0.000206,0.000340,0.000400,0.000393,0.000570,0.000280,0.000453,0.000270,0.000224,0.000308,0.000158,0.000073,0.000162,0.000033,0.000085,0.000144,0.000072


### Find the optimal asset allocation

In [120]:
n = len(symbols)
R = 0.001 # 0.1% return
e = np.ones(n)
X = cp.Variable(n)
obj = cp.Minimize(cp.quad_form(X,S)) # computes x^T S x
constraints = [r@X == R]
constraints += [e@X == 1]
constraints += [X >= 0]
problem = cp.Problem(obj, constraints)

In [124]:
problem.solve()
for i in range(14):
    print(symbols[i], " :", np.round(X.value[i], 3))

TSLA  : 0.0
SHOP  : 0.331
SQ  : 0.0
QS  : 0.0
OPEN  : 0.0
MP  : 0.0
COIN  : 0.153
HRI  : 0.0
CLF  : 0.0
LOW  : 0.0
GS  : 0.0
KO  : 0.0
SPGI  : 0.0
GIS  : 0.516


### Add buget constraint

The sum of the assets times its price shouldn't exceed the budget. The budget will be $1000.
$$ \mathbf{p}^T \mathbf{x} \leq 1000 $$
where p is the price of the assets

In [122]:
# get the price of each assets
p = []
for stock in symbols:
    data = yf.Ticker(stock)
    df = data.history(period="5y")
    p.append(df['Close'].iloc[-1])

constraints += [p@X <= 100]
problem = cp.Problem(obj, constraints)

In [123]:
result = problem.solve('ECOS')
for i in range(14):
    print(symbols[i], " :", np.round(X.value[i], 3))

TSLA  : 0.0
SHOP  : 0.331
SQ  : 0.0
QS  : 0.0
OPEN  : 0.0
MP  : 0.0
COIN  : 0.153
HRI  : 0.0
CLF  : 0.0
LOW  : 0.0
GS  : 0.0
KO  : 0.0
SPGI  : 0.0
GIS  : 0.516
